In [2]:
import numdifftools as nd
import math
import numpy as np
import bisect
import pandas as pd
import collections
from math import log



Q1 gradient descent


In [3]:
def function(x):
    return math.pow(x[0],4) + 4*x[0]*x[1] + 2 * x[1] + (x[1]**2)/2

def grad(x,func):
    return nd.Gradient(func)(x)


def gradient_descentA(f, grad_f, eta, x_0, max_iter=100):
    for i in range(max_iter):
        x_t = x_0 - eta(i)* grad_f(x_0,f)
        x_0 = x_t
    return x_t

def gradient_descentB(f, grad_f, eta, x_0, max_iter=100):
    for i in range(max_iter):
        x_t = x_0 - eta(i)* grad_f(x_0,f)
        x_0 = x_t
    return x_t

def gradient_descentC(f,grad_f, eta,c,eta_start, x_0, milestones,max_iter=100):
    for i in range(max_iter):
        x_t = x_0 - eta(i,milestones,c,eta_start)* grad_f(x_0,f)
        x_0 = x_t
    return x_t

def eta_const(t,c = 0.01):
    return c 

def eta_sqrt(t, c = 0.1 ):
    return c/np.sqrt(t+1)

def eta_multistep(t, milestones, c, eta_init = 1):
    interval = len(milestones) + 1
    query_index = bisect.bisect_left(milestones, t)
    res = [eta_init*math.pow(c,i) for i in range(interval)]
    return res[query_index]



x_0 = [1,1]
A = gradient_descentA(f=function,grad_f=grad,eta=eta_const,x_0=x_0)
B = gradient_descentB(f=function,grad_f=grad,eta=eta_sqrt,x_0=x_0)
C = gradient_descentC(f=function,grad_f=grad,c=0.5,milestones=[10,60,90],eta_start=0.1,eta=eta_multistep ,x_0=x_0)

valueA = function(A)
valueB = function(B)
valueC = function(C)



print(valueA)
print(valueB)
print(valueC)




-16.087776514949773
-27.467055951442692
-34.69816537692322


Q2 coordinate descent

In [4]:
x=[5,10,5]

def argmin_xi(x):
    x[0] = (0.5*x[1])**(1/3)
    x[1] = 0.5*(x[0] - x[2])
    x[2] = -0.5*x[1]
    result = [x[0],x[1],x[2]]
    return result

def coordinate_descent(argmin,x_0,max_iter):
    for i in range(max_iter):
        x_1 = argmin(x_0)
        x_0 = x_1
    return x_1



x1,x2,x3 = coordinate_descent(argmin=argmin_xi,x_0=x,max_iter=100)
print(x1)
    



(0.5773502691896257+2.2237664939066243e-30j)


Q3 confusion matrix

In [5]:
def function(x):
    return -0.078*x[0] - 0.227*x[1] + 0.165

def confusion(l1,l2):  # l2: predicted, l1: ground truth
    l1 = np.array(l1)
    l2 = np.array(l2)
    TP = sum(((l2 == 1) & (l1 == 1)) == 1).item()
    FN = sum(((l2 == 0) & (l1 == 1)) == 1).item()
    FP = sum(((l2 == 1) & (l1 == 0)) == 1).item()
    TN = sum(((l2 == 0) & (l1 == 0)) == 1).item()
    return TP, TN, FP, FN


x1 = [3,-2,2,-1.5,1.5,-2.5,-2,2]
x2 = [1,2,2,2,3.5,2.5,0,2.5]
y = [0,1,0,1,0,1,1,0]
X = np.stack((x1, x2), axis=1)

val = [function(X[i]) for i in range(len(X))]
pred = np.array([bisect.bisect_left([0], i) for i in val])
TP,TN,FP,FN = confusion(l1=y,l2=pred)
print(TP)
print(TN)
print(FP)
print(FN)


1
4
0
3


Q4 naive bayes

In [6]:



class naive_bayes_clf:
    def __init__(self):
        self.p_y = None
        self.cpt_0 = None
        self.cpt_1 = None

    @staticmethod
    def class_probability(data: pd.DataFrame) -> float:
        """
        p(y) calculation
        """
        return data["target"].sum() / data.shape[0]

    def conditional_probability(self, data: pd.DataFrame, feature: str, feature_value: int):
        """
        p(x|y) calculation given conditional independence
        """
        cf_0, cf_1 = data[data["target"] == 0], data[data["target"] == 1]

        # Calculate probability of a feature having a value, given target 0 or 1, respectively.
        prob_0 = cf_0[cf_0[feature] == feature_value][[feature]].count() / cf_0[[feature]].count()
        prob_1 = cf_1[cf_1[feature] == feature_value][[feature]].count() / cf_1[[feature]].count()
        f_index = list(data.columns).index(feature)
        if f_index == 2:
            feature_value -= 1

        # Save conditional probability of feature value to the right indices
        self.cpt_0[f_index][feature_value], self.cpt_1[f_index][feature_value] = prob_0[0], prob_1[0]

    def train(self, data: pd.DataFrame):
        """
        Train naive_bayes clf by calculating p(y) and p(x|y)
        """
        self.p_y = self.class_probability(data)
        self.cpt_0 = [[0 for _ in data[col].unique()] for col in data.columns[:-1]]
        self.cpt_1 = [[0 for _ in data[col].unique()] for col in data.columns[:-1]]
        for col in data.columns[:-1]:
            for val in data[col].unique():
                self.conditional_probability(data, col, val)

        return self.p_y, self.cpt_0, self.cpt_1

    def predict(self, vector: list) -> int:
        """
        Predict outcome of vector by p(y) * ∏ p(x|y)
        """
        assert all([val is not None for val in [self.p_y, self.cpt_0, self.cpt_1]]), "First train the model."
        prob_0, prob_1 = 1, 1
        vector[2] -= 1

        # p(x|y) multiplication for conditional independence
        for feature, value in zip(self.cpt_0, vector):
            prob_0 *= feature[value]

        for feature, value in zip(self.cpt_1, vector):
            prob_1 *= feature[value]

        # multiply by p(y) to get p(y|x)
        prob_0 *= 1 - self.p_y
        prob_1 *= self.p_y

        return 0 if prob_0 > prob_1 else 1


if __name__ == '__main__':
    train_data = pd.read_csv(r'heart_train_data.csv')
    validation_data = pd.read_csv(r'heart_validate_data.csv')

    model = naive_bayes_clf()
    class_prob, cpt_0, cpt_1 = model.train(train_data)

    # 5A
    result = model.predict(vector=[1, 1, 2])
    print(f"Patient diagnosis is: {'No disease' if result == 1 else 'Disease'}.")

    # 5B
    atypical_angina = train_data.cp.value_counts()[1] / sum(train_data.cp.value_counts())
    ex_ang = train_data.exang.value_counts()[1] / sum(train_data.exang.value_counts())
    tha_l = train_data.thal.value_counts()[1] / sum(train_data.thal.value_counts())
    abcde = atypical_angina+ex_ang+tha_l
    print(f"{round(abcde*100, 3)}% of the people in the training data has heart disease.")

    # 5C
    print(f"The probability of having a normal thallium heart scan given that the patient has heart disease "
          f"equals to {round(cpt_0[2][0], 3)}.")

    # 5D
    x_validate = validation_data.iloc[:, :-1]
    y_true = list(validation_data.iloc[:, -1])
    y_pred = [model.predict(vector=row[1]) for row in x_validate.iterrows()]

    accuracy = sum(1 for x, y in zip(y_true, y_pred) if x == y) / len(y_true)

    print(f"The accuracy of the model is {round(accuracy, 3)}.")

Patient diagnosis is: No disease.
53.81% of the people in the training data has heart disease.
The probability of having a normal thallium heart scan given that the patient has heart disease equals to 0.075.
The accuracy of the model is 0.747.


Q5 decision tree

In [7]:


train_dataset = pd.read_csv(r'heart_train_data.csv')
validate_dataset = pd.read_csv(r'heart_validate_data.csv')
dataset = train_dataset.values.tolist()
validate_dataset = validate_dataset.values.tolist()
impurity = []
labels = ['cp','exang','thal']
targetLabels = ['a heart disease',' no heart disease']


def calEntropy(dataset):
    #Calculate entropy which is used as impurity measure
    y_data = [i[-1] for i in dataset]
    entropy = 0
    count = collections.Counter(y_data)
    #Formula entropy=sum(-log(p,2)*p)
    for i in range(len(count)-1):
        p = count[i]/len(y_data)
        entropy -= p * log(p, 2)
    return  entropy

def splitDataset(dataset, feature, value):
    #split the dataset set into two parts according to the selected features and values
    subdataset1 = []
    subdataset2 = []
    # subdataset1 with the feature equals to value
    # subdataset2  with the frature not equals to value
    for featVec in dataset: 
        if featVec[feature] == value:
            subdataset1.append(featVec)
        else:
            subdataset2.append(featVec)
    return  subdataset1,subdataset2

def bestFeature(dataset):
    #Calculate the bestway to split the dataset by comparing the entropy.
    numFeatures = len(dataset[0]) - 1
    bestEntropy = 999
    bestFeature = -1
    bestValue = -1
    # Traverse all the features and values to splite the data. 
    # And calculate the entropy of each way of spliting to find the lowest entropy.
    for i in range(numFeatures):
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)
        uniqueVals = list(uniqueVals)
        entropy=0.0
        for value in uniqueVals[0:len(uniqueVals)]:
            subdataset1,subdataset2 = splitDataset(dataset, i, value)
            p = len(subdataset1) / float(len(dataset))
            entropy = p * calEntropy(subdataset1) + (1-p)*calEntropy(subdataset2)
            if (entropy < bestEntropy):
                bestEntropy = entropy
                bestFeature = i
                bestValue = value
    return bestFeature,bestValue

def calLeaf(dataset):
    #determine the node value based on the majority of data
    y_data = [i[-1] for i in dataset]
    count = collections.Counter(y_data)
    if count[0]>count[1]:
        impurity.append(1-count[0]/len(y_data))
        return 0
    else:
        impurity.append(1-count[1]/len(y_data))
        return 1

In [8]:
class cartTree(object):
    def __init__(self,dataset = None,depth = 0):
        #Each node represents a choice. 
        #Feature and value is the way to choose.
        #Left and right represents the data that has been splited by the choice. 
        #Depth is the layer of ndoe to determine the termination conditions.
        self.dataset = dataset
        self.left = None
        self.right = None
        self.feature = -2
        self.value = -2
        self.depth = depth
    
    def creatTree(self,dataset,depth):
        #Creat the decision tree.
        self.dataset = dataset
        feature = -1
        value = -1
        self.depth = depth
        if self.depth < 3:
            feature,value = bestFeature(dataset)
            self.feature = feature
            self.value = value
            subdataset1,subdataset2 = splitDataset(dataset, feature, value)
            #Recurse on child nodes
            self.left = cartTree(subdataset1,depth+1)
            self.left.creatTree(subdataset1,depth+1)
            self.right = cartTree(subdataset2,depth+1)
            self.right.creatTree(subdataset2,depth+1)
        else:
            self.value = calLeaf(dataset)
            self.feature = -1
    
    def calAccuracy(self,dataset):
        #Count the number of correct predictions in the validate_dataset.
        feature = self.feature
        value = self.value
        sumRight = 0
        if feature != -1:
            #If it is not a leaf node, calculate the sum of left and right.
            subdataset1,subdataset2 = splitDataset(dataset, feature, value)
            sum1 = self.left.calAccuracy(subdataset1)
            sum2 = self.right.calAccuracy(subdataset2)
            sumRight = sum1 + sum2
            return sumRight
        else:
             #If it is a leaf node, return the number of correct predictions due to the value of leaf node.
            y_data=[i[-1] for i in dataset]
            count = collections.Counter(y_data)
            if self.value == 0:
                return count[0]
            else:
                return count[1]
            
    def calPatient(self,pdata):
        #calculate the result of a certain patient
        node = cartTree() 
        node = self
        for i in pdata:
            if i == self.value:
                node = node.left
            else:
                node = node.right
        return node.value

In [9]:
impurity = []
tree = cartTree()
tree.creatTree(dataset,0)
# A
right = tree.calAccuracy(validate_dataset)
accuracy = right/len(validate_dataset)
print('accuracy = %f ' %accuracy)
# B
rootFeature,rootValue = bestFeature(dataset)
print('The decision of the root of the tree is based on %s , value = %d  ' %(labels[rootFeature], rootValue))
# C
patient = [1,1,2]
result = tree.calPatient(patient)
print('This patient has %s' %targetLabels[result])
# D
print('The impurity of each leaf is')
for i in impurity:
    print('    %f' %i)


accuracy = 0.725275 
The decision of the root of the tree is based on cp , value = 0  
This patient has  no heart disease
The impurity of each leaf is
    0.347826
    0.200000
    0.181818
    0.048780
    0.368421
    0.333333
    0.222222
    0.064103
